In [2]:
import warnings
warnings.filterwarnings("ignore")
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human" )
env = JoypadSpace(env, SIMPLE_MOVEMENT)

env.observation_space.shape

(240, 256, 3)

In [ ]:
# hypers 
lr = 0.001
lambdaa = 0.99
gamma = 0.99
epochs = 10
batchsize = 1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

class policy(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.LazyConv2d(1,(1,1))
        #self.conv1 = nn.LazyConv2d()
        #self.conv1 = nn.LazyConv2d()
        #self.conv1 = nn.LazyConv2d()
        #self.conv1 = nn.LazyConv2d()

        self.l1 = nn.LazyLinear()
        self.l2 = nn.LazyLinear()
        self.output = nn.LazyLinear()

        self.optim = Adam(self.parameters(),lr=lr)
        
    def forward(self,x):
        x = self.input(x)
        return F.softmax(x,-1)
    
class value(nn.Module):
    def __init__(self):
        self.input = nn.LazyConv2d()
        #self.conv1 = nn.LazyConv2d()
        #self.conv1 = nn.LazyConv2d()

        self.l1 = nn.LazyLinear()
        self.output = nn.LazyLinear()

        self.optim = Adam(self.parameters(),lr=lr)
        super().__init__()
        pass
        

    def forward(self):
        pass
 

torch.Size([1, 256, 3])